In [2]:
import os
import time
import numpy as np
from IPython.display import clear_output
import plotly.graph_objects as go
import plotly.express as px
import brainflow
from brainflow.data_filter import DataFilter, FilterTypes, AggOperations
from brainflow.board_shim import BoardShim, BrainFlowInputParams, BoardIds, BrainFlowError
from matplotlib.pyplot import figure

from tqdm.notebook import tqdm
import datetime


from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
import itertools
from sklearn.utils import shuffle
from scipy import signal

from utils.svm import DCFilter
from utils.visualize import Vis, VisAction

%matplotlib qt

from utils.svm import preProcess_1, preProcess_buffer

%load_ext autoreload
%autoreload 2


C:\Users\Nemes\anaconda3\envs\tf\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\Nemes\anaconda3\envs\tf\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
C:\Users\Nemes\anaconda3\envs\tf\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
C:\Users\Nemes\anaconda3\envs\tf\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  stacklevel=1)


In [3]:
####      INIT BOARD        #######
BoardShim.enable_dev_board_logger()
params = BrainFlowInputParams()
board = BoardShim(BoardIds.MINDROVE_WIFI_BOARD, params)
print(board.get_eeg_channels(board_id=BoardIds.MINDROVE_WIFI_BOARD))

try:
    board.stop_stream()
    board.release_session()
except:
    ...
    
board.prepare_session()
sample_rate = board.get_sampling_rate(16)
n_channels  = 6

print("Device ready (sampling rate: {}hz)".format(sample_rate))

[0, 1, 2, 3, 4, 5, 6, 7]
Device ready (sampling rate: 500hz)


In [4]:
sample_rate = 500
num_points = int(sample_rate * 1)
input_length = 100
board.start_stream(500*60) # 1 MIN


std_threshold =100
std_threshold_max = 2000
prepare_steps = 50

action_time = None
print("Preparing...")


for i in range(1000): # sample_rate
    data = board.get_current_board_data(num_points)
    time.sleep(0.1)
    #data = data[:6,:sample_rate] #keep the data of the eeg channels only, and remove data over the trial length
    data = data[:6,:]
    data = DCFilter(data)

    if data.shape[1] == 500:
        current = data[:, -500:]
        print(current.shape)
        current = signal.resample(current, input_length, axis=-1)
        print(current.shape)
        min, max, std = Vis(current, [-1000,1000])



Preparing...
(6, 500)
(6, 100)
(6, 500)
(6, 100)
(6, 500)
(6, 100)
(6, 500)
(6, 100)
(6, 500)
(6, 100)
(6, 500)
(6, 100)
(6, 500)
(6, 100)
(6, 500)
(6, 100)
(6, 500)
(6, 100)
(6, 500)
(6, 100)
(6, 500)
(6, 100)
(6, 500)
(6, 100)
(6, 500)
(6, 100)
(6, 500)
(6, 100)
(6, 500)
(6, 100)
(6, 500)
(6, 100)
(6, 500)
(6, 100)
(6, 500)
(6, 100)
(6, 500)
(6, 100)
(6, 500)
(6, 100)
(6, 500)
(6, 100)
(6, 500)
(6, 100)
(6, 500)
(6, 100)
(6, 500)
(6, 100)
(6, 500)
(6, 100)
(6, 500)
(6, 100)
(6, 500)
(6, 100)
(6, 500)
(6, 100)
(6, 500)
(6, 100)
(6, 500)
(6, 100)
(6, 500)
(6, 100)
(6, 500)
(6, 100)
(6, 500)
(6, 100)
(6, 500)
(6, 100)
(6, 500)
(6, 100)
(6, 500)
(6, 100)
(6, 500)
(6, 100)
(6, 500)
(6, 100)
(6, 500)
(6, 100)
(6, 500)
(6, 100)
(6, 500)
(6, 100)
(6, 500)
(6, 100)
(6, 500)
(6, 100)
(6, 500)
(6, 100)
(6, 500)
(6, 100)
(6, 500)
(6, 100)
(6, 500)
(6, 100)
(6, 500)
(6, 100)
(6, 500)
(6, 100)
(6, 500)
(6, 100)
(6, 500)
(6, 100)
(6, 500)
(6, 100)
(6, 500)
(6, 100)
(6, 500)
(6, 100)
(6, 500)
(6, 10

KeyboardInterrupt: 

In [ ]:
sample_rate = 500
num_points = int(sample_rate * 1)
input_length = 100
board.start_stream(450000)


std_threshold =100
std_threshold_max = 2000
prepare_steps = 50

action_time = None
print("Preparing...")
for i in range(1000): # sample_rate
    data = board.get_current_board_data(num_points*60)
    data = data[:6,:sample_rate] #keep the data of the eeg channels only, and remove data over the trial length
    # print(data.shape)
    # time.sleep(0.01)
    
    data = preProcess_1(data, input_length)
    
    min, max, std = Vis(data)
    # clear_output()
    # print(std)

    if i == prepare_steps:
        print("Starting....")

    if i > prepare_steps:
        if action_time is None:
            if std > std_threshold:
                action_time  =  datetime.datetime.now()
                stop_time = action_time + datetime.timedelta(milliseconds=500)
        else:
            ## IF TOO MUCH NOISE IN THE SIGNAL 
            ## DONT START THE ACTION
            if std > std_threshold_max:
                #print("Noise")
                action_time = None 
            time_diff = (datetime.datetime.now() - stop_time).total_seconds()
            if abs(time_diff) < 0.1:
                VisAction(data)
                action_time = None
